In [1]:
import pandas as pd
import numpy as np
import re

import requests
from bs4 import BeautifulSoup

from selenium import webdriver as wd
import time

import pickle

from selenium.common.exceptions import NoSuchElementException

In [2]:
path = "C:/Users/wldnf/Downloads/chromedriver_win32/chromedriver.exe" #자신의 chromedriver 위치 할당

## Final

1. 연도별 링크 뽑기

In [3]:
driver = wd.Chrome(path)
url = 'https://music.bugs.co.kr/years'
driver.get(url)

In [4]:
# 성인인증이 필요한 노래 가사를 위하여 로그인
driver.find_element_by_id('loginHeader').find_element_by_tag_name('a').click()
driver.find_element_by_id('to_bugs_login').click()
driver.find_element_by_name('user_id').send_keys('') # 벅스 아이디 입력
driver.find_element_by_name('passwd').send_keys('') # 벅스 비밀번호 입력
driver.find_element_by_class_name('submit').click()

In [5]:
# 연도별 접근 링크 뽑기
year_link_list = []

infos = driver.find_elements_by_class_name("info")
for info in infos:
    link = info.find_element_by_tag_name("a").get_attribute('href')
    year_link_list.append(link)
year_link_list

['https://music.bugs.co.kr/musicpd/albumview/30871',
 'https://music.bugs.co.kr/musicpd/albumview/25698',
 'https://music.bugs.co.kr/musicpd/albumview/20440',
 'https://music.bugs.co.kr/musicpd/albumview/15389',
 'https://music.bugs.co.kr/musicpd/albumview/11152',
 'https://music.bugs.co.kr/musicpd/albumview/7479',
 'https://music.bugs.co.kr/musicpd/albumview/6683',
 'https://music.bugs.co.kr/musicpd/albumview/6684',
 'https://music.bugs.co.kr/musicpd/albumview/6685',
 'https://music.bugs.co.kr/musicpd/albumview/6709',
 'https://music.bugs.co.kr/musicpd/albumview/6710',
 'https://music.bugs.co.kr/musicpd/albumview/6712',
 'https://music.bugs.co.kr/musicpd/albumview/6713',
 'https://music.bugs.co.kr/musicpd/albumview/6715',
 'https://music.bugs.co.kr/musicpd/albumview/6716',
 'https://music.bugs.co.kr/musicpd/albumview/6717',
 'https://music.bugs.co.kr/musicpd/albumview/6718',
 'https://music.bugs.co.kr/musicpd/albumview/6719',
 'https://music.bugs.co.kr/musicpd/albumview/6720',
 'https

2. 연도별 링크에서 곡별 정보 뽑기

In [6]:
year_list = []
title_list = []
singer_list = []
time_list = []
lyric_list = []
singer_type_list = []

In [7]:
for year_link in year_link_list:
    # 연도별 TOP100 페이지로 이동
    driver.get(year_link)
    time.sleep(2)
    
    # 연도
    year = driver.find_element_by_id('hyrendContentBody').find_element_by_tag_name('h1').text
    # 연도별 곡 개수
    innercontainer4_tbody = driver.find_elements_by_class_name('innerContainer')[3].find_element_by_tag_name('tbody')
    innercontainer4_trs = innercontainer4_tbody.find_elements_by_tag_name('tr')
    year_song_num = len(innercontainer4_trs)
    for i in range(year_song_num):
        year_list.append(year)
    
    # 해당 연도의 TOP100 곡별 링크 뽑기
    song_url_list = []
    song_trackinfo = innercontainer4_tbody.find_elements_by_class_name('trackInfo')
    for trackinfo in song_trackinfo:
        song_url_list.append(trackinfo.get_attribute('href'))
    
    # 곡별 링크에서 관련 정보 뽑기
    for song_url in song_url_list:
        driver.get(song_url)
        time.sleep(2)
    
        # 타이틀
        title = driver.find_element_by_id('hyrendContentBody').find_element_by_tag_name('h1').text
        title_list.append(title)
    
        # 가사
        try:
            lyric = driver.find_element_by_class_name('lyricsContainer').find_element_by_tag_name('xmp').text
            lyric_list.append(lyric)
        except NoSuchElementException:
            lyric = 'NA'
            lyric_list.append(lyric)
    
        # 가수
        song_info = driver.find_element_by_class_name('info')
        singer = song_info.find_element_by_tag_name('td').text
        singer_list.append(singer)
    
        # 시간
        runtime = song_info.find_element_by_tag_name('time').text
        time_list.append(runtime)
    
        # 솔로/그룹(성별)
        type_url = song_info.find_element_by_tag_name('td').find_element_by_tag_name('a').get_attribute('href')
        driver.get(type_url)
        time.sleep(2)
    
        singer_type = driver.find_element_by_class_name('info').find_element_by_tag_name('td').text
        singer_type_list.append(singer_type)
    
    

In [9]:
print(len(title_list),len(year_list),len(singer_list),len(time_list),len(singer_type_list),len(lyric_list))

5146 5146 5146 5146 5146 5146


In [28]:
from collections import Counter

Counter(singer_type_list) # 그룹유형이 대한민국/미국으로 크롤링된 경우가 있음

Counter({'그룹 (남성)': 703,
         '듀오 (남성)': 281,
         '솔로 (남성)': 2100,
         '그룹 (여성)': 318,
         '솔로 (여성)': 1394,
         '듀오 (여성)': 83,
         '그룹 (혼성)': 162,
         '듀오 (혼성)': 51,
         '그룹': 5,
         '솔로': 15,
         '대한민국': 32,
         '미국': 2})

추가) 앨범별 장르 뽑기

In [3]:
driver = wd.Chrome(path)
url = 'https://music.bugs.co.kr/years'
driver.get(url)

In [5]:
genre_list = []

In [6]:
for year_link in year_link_list:
    # 연도별 TOP100 페이지로 이동
    driver.get(year_link)
    time.sleep(1)
    
    
    # 해당 연도의 TOP100 곡별 링크 뽑기
    song_url_list = []
    innercontainer4_tbody = driver.find_elements_by_class_name('innerContainer')[3].find_element_by_tag_name('tbody')
    song_trackinfo = innercontainer4_tbody.find_elements_by_class_name('trackInfo')
    for trackinfo in song_trackinfo:
        song_url_list.append(trackinfo.get_attribute('href'))
    
    # 곡별 링크에서 관련 정보 뽑기
    for song_url in song_url_list:
        driver.get(song_url)
        time.sleep(1)
        # 장르
        song_info = driver.find_element_by_class_name('info')
        song_info_trs = song_info.find_elements_by_tag_name('tr')

        for tr in song_info_trs:
            if tr.find_element_by_tag_name('th').text=='앨범':
                album_tr = tr
            else:
                pass
    
        album_url = album_tr.find_element_by_tag_name('td').find_element_by_tag_name('a').get_attribute('href')
        driver.get(album_url)
        time.sleep(1)

        album_info = driver.find_element_by_class_name('info')
        album_info_trs = album_info.find_elements_by_tag_name('tr')

        for tr in album_info_trs:
            if tr.find_element_by_tag_name('th').text=='장르':
                genre = tr.find_element_by_tag_name('td').text
                genre_list.append(genre)
            else:
                pass
    
    
       

In [7]:
len(genre_list)

5146

## 각 정보 pickle 파일로 저장

In [29]:
## Save pickle
# year
with open("year.pickle","wb") as fw:
    pickle.dump(year_list, fw)
# title
with open("title.pickle","wb") as fw:
    pickle.dump(title_list, fw)
# singer 
with open("singer.pickle","wb") as fw:
    pickle.dump(singer_list, fw)
# runtime
with open("runtime.pickle","wb") as fw:
    pickle.dump(time_list, fw) 
# type    
with open("type.pickle","wb") as fw:
    pickle.dump(singer_type_list, fw)
# lyric
with open("lyric.pickle","wb") as fw:
    pickle.dump(lyric_list, fw)
# genre
with open("genre.pickle","wb") as fw:
    pickle.dump(genre_list, fw)   

## 각 정보 pickle 파일로 불러오기

In [10]:
## Load pickle
# year
with open("year.pickle","rb") as fr:
    year_list = pickle.load(fr)

# title
with open("title.pickle","rb") as fr:
    title_list = pickle.load(fr)

# singer
with open("singer.pickle","rb") as fr:
    singer_list = pickle.load(fr)

# runtime
with open("runtime.pickle","rb") as fr:
    time_list = pickle.load(fr)

# type
with open("type.pickle","rb") as fr:
    singer_type_list = pickle.load(fr)

# singer
with open("lyric.pickle","rb") as fr:
    lyric_list = pickle.load(fr)

# genre
with open("genre.pickle","rb") as fr:
    genre_list = pickle.load(fr)

## 데이터프레임 만들기

In [11]:
bugs_df = pd.DataFrame({'year':year_list,'title':title_list, 'singer':singer_list, 'genre':genre_list,
                        'runtime':time_list, 'type':singer_type_list,'lyric':lyric_list})
bugs_df

,year,title,singer,genre,runtime,type,lyric
0,2018년 베스트 가요 콜렉션,사랑을 했다 (LOVE SCENARIO),iKON,랩/힙합,03:29,그룹 (남성),사랑을 했다 우리가 만나\n지우지 못할 추억이 됐다\n볼만한 멜로드라마\n괜찮은 결...
1,2018년 베스트 가요 콜렉션,그날처럼,장덕철,발라드/댄스/팝,03:52,그룹 (남성),참 많은 시간이 흘러가고\n넌 어떻게 사는지 참 궁금해\n날 걱정하는 사람들에게\n...
2,2018년 베스트 가요 콜렉션,선물,멜로망스(MeloMance),"인디, 발라드/댄스/팝",04:20,듀오 (남성),빛이 들어오면 자연스레 뜨던 눈 \n그렇게 너의 눈빛을 보곤\n사랑에 눈을 떴어\n...
3,2018년 베스트 가요 콜렉션,"모든 날, 모든 순간 (Every day, Every Moment)",폴킴(Paul Kim) CONNECT 아티스트,"발라드/댄스/팝, OST",03:30,솔로 (남성),네가 없이 웃을 수 있을까\n생각만 해도 눈물이나\n힘든 시간 날 지켜준 사람\n이...
4,2018년 베스트 가요 콜렉션,뚜두뚜두 (DDU-DU DDU-DU),BLACKPINK,발라드/댄스/팝,03:29,그룹 (여성),착한 얼굴에 그렇지 못한 태도\n가녀린 몸매 속 가려진 Volume은 두 배로\n거...
5,2018년 베스트 가요 콜렉션,뿜뿜,모모랜드(MOMOLAND),발라드/댄스/팝,03:28,그룹 (여성),설레어 너와 나의 랑데뷰 (랑데뷰)\n내 마음을 들었다 놨다 해 맘대루 (맘대루)\...
6,2018년 베스트 가요 콜렉션,You,멜로망스(MeloMance),알앤비/소울,04:32,듀오 (남성),어떻게 난 해야 해 니가 떠난다고 말하면\n난 혼자 남겨지는 게 더 두려울 텐데\n...
7,2018년 베스트 가요 콜렉션,그때 헤어지면 돼,로이킴,발라드/댄스/팝,04:08,솔로 (남성),나를 사랑하는 법은 어렵지 않아요\n지금 모습 그대로 나를 꼭 안아주세요\n우리 나...
8,2018년 베스트 가요 콜렉션,Roller Coaster,청하,발라드/댄스/팝,03:32,솔로 (여성),I remember very first time in love\nI remember...
9,2018년 베스트 가요 콜렉션,instagram,DEAN(딘),알앤비/소울,04:15,솔로 (남성),내일이 올 걸 아는데\n난 핸드폰을 놓지 못해\n잠은 올 생각이 없대 yeah\n다...


In [12]:
bugs_df.to_csv('Bugs.csv', index=False)

In [13]:
bugs = pd.read_csv('Bugs.csv')
bugs[1000:1010]

,year,title,singer,genre,runtime,type,lyric
1000,2008년 베스트 가요 콜렉션,주문 - MIROTIC,동방신기 (TVXQ!),발라드/댄스/팝,03:28,그룹 (남성),"시작은 달콤하게, 평범하게 나에게 끌려 \r\n언제나 그랬듯이 먼저 말을 걸어와 \..."
1001,2008년 베스트 가요 콜렉션,10점 만점에 10점,2PM,발라드/댄스/팝,03:22,그룹 (남성),That girl 너무 예뻐 Oh my god 기가 막혀 \r\n이건 본 적이 없는...
1002,2008년 베스트 가요 콜렉션,1분 1초 (feat. 타루),에픽하이 (EPIK HIGH),랩/힙합,04:28,그룹 (남성),I can't let go (go go go)\r\n어딜봐도 니 모습이 보이고 무너...
1003,2008년 베스트 가요 콜렉션,Baby Baby,소녀시대 (GIRLS' GENERATION),발라드/댄스/팝,03:11,그룹 (여성),떨리는 맘 고마운 맘\r\n가득히 담아서\r\n젤 예쁜 포장 속에\r\n전하고 싶은...
1004,2008년 베스트 가요 콜렉션,Bad Boy,손담비,"일렉트로닉, 발라드/댄스/팝",03:23,솔로 (여성),처음엔 모든 걸 다 해주더니 \r\n내 마음을 흔들어 놓고 \r\n조금씩 나에게 더...
1005,2008년 베스트 가요 콜렉션,Beautiful Life,V.O.S.,"발라드/댄스/팝, 알앤비/소울",03:25,그룹 (남성),변해버린 내 모습에 놀라 \r\n잘라버린 긴 머리 끝이\r\n이렇게도 꽤나 내가 괜...
1006,2008년 베스트 가요 콜렉션,D.I.S.C.O (feat. TOP),엄정화,"일렉트로닉, 발라드/댄스/팝",03:25,솔로 (여성),*\r\nD.I.S.C.O 미친 듯이 춤추고 \r\n네 멋대로 Do the disc...
1007,2008년 베스트 가요 콜렉션,Dangerous,은지원,랩/힙합,03:29,솔로 (남성),언제나 또 넌 내게 많은 것을 원해\r\n내가 할수 있는건 다 했는데\r\nI wa...
1008,2008년 베스트 가요 콜렉션,Hey Mr. Big,이효리,발라드/댄스/팝,03:11,솔로 (여성),Hey listen Mr. boy \r\n잘난 척 하는 입술로\r\n이제는 속 깊은...
1009,2008년 베스트 가요 콜렉션,Hot Girl,씨야(SeeYa),발라드/댄스/팝,03:15,그룹 (여성),@\r\nJust one more time \r\nJust one more time...
